# M5 - Simple FEのまとめ
https://www.kaggle.com/kyakovlev/m5-simple-fe

## ライブラリのimport

In [1]:
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
#メモリの残量の表示
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 
#データサイズの表示      
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
#データ量を抑えることのできる関数
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## 2つのデータをmergeさせる関数
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [5]:
#使い回す変数の指定
TARGET = 'sales'         # Our main target
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

In [6]:
print('Load Main Data')

# データの読み込み
train_df = pd.read_csv('sales_train_evaluation.csv')
prices_df = pd.read_csv('sell_prices.csv')
calendar_df = pd.read_csv('calendar.csv')

Load Main Data


### python melt  
http://nekoyukimmm.hatenablog.com/entry/2015/04/02/115045  
２つ以上の値を格納している列columnを合体して、 １つの値valuesを格納している列columnを作成し、さらに、それに対応した認識子identifier variable(すなわち、もとの列名headerを)格納した列columnを作る。

In [7]:
print('Create Grid')

# d_を1つの列にまとめる。'sales'も対応をさせる
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)




print('Train rows:', len(train_df), len(grid_df))
# 出力した結果をいれるためのdataframeを作成
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])
grid_df = grid_df.reset_index(drop=True)

# 一時的に作成したデータの削除
del temp_df, add_grid,train_df

# メモリの使用量の確認
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))




# index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
#文字列のデータをカテゴリデータに変換をしている
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# カテゴリデータに変換をした上で再度メモリの使用量を確認
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 59181090
    Original grid_df:   3.6GiB
     Reduced grid_df:   1.3GiB


In [8]:
#商品ごとにいつ発売されてからの経過時間を調べる。
print('Release week')

#もし発売されていないなら削除することができるのでメモリの節約にもなる

# Prices are set by week
# so it we will have not very accurate release week 
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()
release_df.columns = ['store_id','item_id','release']

# Now we can merge release_df
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

# 発売をされていないデータを削除したいので、カレンダーデータの一部とつきあわせる
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                      
# Now we can cutoff some rows and safe memory 
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# データの使用量を計算
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# 'release'のデータ量を減らすために0からの表示変更をする
#もともとはいつ発売されたかのdateが値として入っていた。
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# データの使用量を再度計算する
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Release week
    Original grid_df:   1.8GiB
     Reduced grid_df:   1.5GiB


In [9]:
#いったんデータを保存する
print('Save Part 1')
#csvと比べて、pickleデータで保存をする方がメモリの節約になる
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Size: (47735397, 10)


In [10]:
#料金データについて特徴量を作成していく
print('Prices')

# 基本統計量の算出を行う
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# max scalingを行う→どういった効果があるのかは謎
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

# インフレ傾向にある商品もいくつかあるので価格の種類の多さも特徴量に加える
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations
# but would like months and years as "window"
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# 月、年で平均値をとり変化率を調べる
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del prices_df['month'], prices_df['year']

Prices


In [11]:
#料金データを加えたところで再度データの保存
print('Merge prices and save part 2')

# 上記で作成した料金データをmergeする
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# 2回目のデータの保存
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)
del prices_df

# We can remove new columns
# or just load part_1
grid_df = pd.read_pickle('grid_part_1.pkl')

Merge prices and save part 2
Mem. usage decreased to 1822.44 Mb (62.2% reduction)
Size: (47735397, 13)


## python 時系列のデータ処理
https://qiita.com/motoki1990/items/8275dbe02d5fd5fa6d2d

In [12]:
#カレンダーデータを処理をしていく
#MAIN_INDEX = ['id','d']
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# 文字列から時系列データに変更を行う
grid_df['date'] = pd.to_datetime(grid_df['date'])

# dateをもとに基本的な時系列に関する特徴量を作成をしていく
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']

In [13]:
#カレンダーデータを結合させる
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

Save part 3
Size: (47735397, 16)


In [14]:
########################### Some additional cleaning
#################################################################################

## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

In [15]:
#SimpleFEでやってきたことのまとめ

# 3つのデータをつなげて出力をしてみる
grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)
                     
# メモリの使用量の確認
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print('Size:', grid_df.shape)
print(grid_df.info())

           Full Grid:   2.5GiB
Size: (47735397, 34)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47735397 entries, 0 to 47735396
Data columns (total 34 columns):
id                  category
item_id             category
dept_id             category
cat_id              category
store_id            category
state_id            category
d                   int16
sales               float64
release             int16
sell_price          float16
price_max           float16
price_min           float16
price_std           float16
price_mean          float16
price_norm          float16
price_nunique       float16
item_nunique        int16
price_momentum      float16
price_momentum_m    float16
price_momentum_y    float16
event_name_1        category
event_type_1        category
event_name_2        category
event_type_2        category
snap_CA             category
snap_TX             category
snap_WI             category
tm_d                int8
tm_w                int8
tm_m                

# Lags features→lags_df_28.pklの作成
https://www.kaggle.com/kyakovlev/m5-lags-features

時系列データを扱う際は必ず使えるものなのでコピペをしてうまく使うべき！

## .assignによる特徴量の追加
https://note.nkmk.me/python-pandas-assign-append/

In [17]:
#simpleFEを元に特徴量を作成する
grid_df = pd.read_pickle('grid_part_1.pkl')
# We need only 'id','d','sales'
# to make lags and rollings
grid_df = grid_df[['id','d','sales']]
SHIFT_DAY = 28
# 変数を作るのにかかった時間の計測
#(time.time() - start_time) / 60)で出力できる
start_time = time.time()

#lag特徴量の作成を行う
print('Create lags')
#lagしたい日を内包表記で表す
LAG_DAYS = [col for col in range(SHIFT_DAY,SHIFT_DAY+15)]
grid_df = grid_df.assign(**{
        '{}_lag_{}'.format(col, l): grid_df.groupby(['id'])[col].transform(lambda x: x.shift(l))
        for l in LAG_DAYS
        for col in [TARGET]
    })
# lag特徴量のメモリ数を制限する
for col in list(grid_df):
    if 'lag' in col:
        grid_df[col] = grid_df[col].astype(np.float16)

print('%0.2f min: Lags' % ((time.time() - start_time) / 60))




# lag特徴量を作成したうえで移動平均を算出する(shiftさせる値は固定)
start_time = time.time()
print('Create rolling aggs')
for i in [7,14,30,60,180]:
    print('Rolling period:', i)
    #移動平均と移動平均の標準偏差を求める
    grid_df['rolling_mean_'+str(i)] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).mean()).astype(np.float16)
    grid_df['rolling_std_'+str(i)]  = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(SHIFT_DAY).rolling(i).std()).astype(np.float16)

# lag特徴量を作成したうえで移動平均を算出する(shiftさせる値もfor文で変えていく)
for d_shift in [1,7,14]: 
    print('Shifting period:', d_shift)
    for d_window in [7,14,30,60]:
        col_name = 'rolling_mean_tmp_'+str(d_shift)+'_'+str(d_window)
        grid_df[col_name] = grid_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(d_shift).rolling(d_window).mean()).astype(np.float16)
    
    
print('%0.2f min: Lags' % ((time.time() - start_time) / 60))

Create lags
8.52 min: Lags
Create rolling aggs
Rolling period: 7
Rolling period: 14
Rolling period: 30
Rolling period: 60
Rolling period: 180
Shifting period: 1
Shifting period: 7
Shifting period: 14
17.52 min: Lags


In [18]:
########################### Export
#################################################################################
print('Save lags and rollings')
grid_df.to_pickle('lags_df_'+str(SHIFT_DAY)+'.pkl')

Save lags and rollings


In [19]:
########################### Final list of new features
#################################################################################
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47735397 entries, 0 to 47735396
Data columns (total 40 columns):
id                        category
d                         int16
sales                     float64
sales_lag_28              float16
sales_lag_29              float16
sales_lag_30              float16
sales_lag_31              float16
sales_lag_32              float16
sales_lag_33              float16
sales_lag_34              float16
sales_lag_35              float16
sales_lag_36              float16
sales_lag_37              float16
sales_lag_38              float16
sales_lag_39              float16
sales_lag_40              float16
sales_lag_41              float16
sales_lag_42              float16
rolling_mean_7            float16
rolling_std_7             float16
rolling_mean_14           float16
rolling_std_14            float16
rolling_mean_30           float16
rolling_std_30            float16
rolling_mean_60           float16
rolling_std_60            float16
ro

# Custom features→mean_encoding_df.pkl
https://www.kaggle.com/kyakovlev/m5-custom-features

このカーネルでは、以下を示します。  
  1.FE作成アプローチ  
  2.順次fe検証  
  3.次元削減  
  4.重要度による特徴量検証  
  5.targetエンコーディング  
  6.作成した特徴量の並列化  

In [22]:
# メモリ使用率を取得
mem = psutil.virtual_memory() 
print(mem.percent)

14.3


In [23]:
# simple FEにて作成をしたもの
grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)

In [24]:
# すべてのデータがいらないのでとりあえず5%だけサンプリングを行う
#idを行列表記にしてidの数()を20個に分割する
keep_id = np.array_split(list(grid_df['id'].unique()), 20)[0]
#いったん上で分割した一つにフォーカスをしたいので、dataframeを作成する
grid_df = grid_df[grid_df['id'].isin(keep_id)].reset_index(drop=True)

# Let's "inspect" our grid DataFrame
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002725 entries, 0 to 3002724
Data columns (total 34 columns):
id                  category
item_id             category
dept_id             category
cat_id              category
store_id            category
state_id            category
d                   int16
sales               float64
release             int16
sell_price          float16
price_max           float16
price_min           float16
price_std           float16
price_mean          float16
price_norm          float16
price_nunique       float16
item_nunique        int16
price_momentum      float16
price_momentum_m    float16
price_momentum_y    float16
event_name_1        category
event_type_1        category
event_name_2        category
event_type_2        category
snap_CA             category
snap_TX             category
snap_WI             category
tm_d                int8
tm_w                int8
tm_m                int8
tm_y                int8
tm_wm               int8

In [25]:
#ベースモデルの作成

# 今後、グローバルVARが必要になります

SEED = 42             # すべてのランダムシード
random.seed(SEED)     # すべてのテストを「確定的」にする
np.random.seed(SEED)
#N_CORESでcpuのコア数を表示することができる
N_CORES = psutil.cpu_count()     # Available CPU cores

TARGET = 'sales'      # Our Target
END_TRAIN = 1941      # And we will use last 28 days as validation

#simple feでtestデータ用に1941以降のデータも入っているが、ここでは絞って分析を行う 
grid_df = grid_df[grid_df['d']<=END_TRAIN].reset_index(drop=True)

# submissionのときに使うデータをまとめたもの
remove_features = ['id','d',TARGET]

# 私たちのベースラインモデルは、新機能のパフォーマンスを迅速にチェックするのに役立ちます
# lightgbmのパラメーターを用意する
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',         # Standart boosting type
                    'objective': 'regression',       # Standart loss for RMSE
                    'metric': ['rmse'],              # as we will use rmse as metric "proxy"
                    'subsample': 0.8,                
                    'subsample_freq': 1,
                    'learning_rate': 0.05,           # 0.5 is "fast enough" for us
                    'num_leaves': 2**7-1,            # We will need model only for fast check
                    'min_data_in_leaf': 2**8-1,      # So we want it to train faster even with drop in generalization 
                    'feature_fraction': 0.8,
                    'n_estimators': 5000,            # We don't want to limit training (you can change 5000 to any big enough number)
                    'early_stopping_rounds': 30,     # モデルのトレーニングはすぐに取りやめる 
                    'seed': SEED,
                    'verbose': -1,
                } 

## 最小二乗法を考える
def rmse(y, y_pred):
    return np.sqrt(np.mean(np.square(y - y_pred)))

#高速機能テストを行うための小さな関数
#estimator = make_fast_test（grid_df）
#将来の分析のためにlgb boosterを返す

#lightgbmで使うdataframeを用意する関数
def make_fast_test(df):
    #remove_featuresは['id','d',TARGET]→lightgbmの変数として使えないので分けていると思う
    #features_columnsはlightgbmに使われる特徴量を集めたものである
    features_columns = [col for col in list(df) if col not in remove_features]

    tr_x, tr_y = df[df['d']<=(END_TRAIN-28)][features_columns], df[df['d']<=(END_TRAIN-28)][TARGET]   
    #訓練データの中の最後の28日間を今分析に用いる。分析結果として出力させたいやつではないってこと
    vl_x, v_y = df[df['d']>(END_TRAIN-28)][features_columns], df[df['d']>(END_TRAIN-28)][TARGET]
    
    train_data = lgb.Dataset(tr_x, label=tr_y)
    valid_data = lgb.Dataset(vl_x, label=v_y)
    #実際にlightgbmで分析を行う
    estimator = lgb.train(
                            lgb_params,
                            train_data,
                            valid_sets = [train_data,valid_data],
                            verbose_eval = 500,
                        )
    
    return estimator

# Make baseline model
baseline_model = make_fast_test(grid_df)

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[255]	training's rmse: 2.85998	valid_1's rmse: 2.39811


### この操作は結局lag1~7までの変数を作成をしているだけ

In [26]:
#通常のラグをテストしてみましょう（7日間）

# poolは並列処理を可能にするライブラリらしいぞ
from multiprocessing import Pool                

##マルチプロセッシングの実行。
#：t_split-ラグ日数のint ＃タイプ：int
#：func-各分割に適用する関数：python関数
##マルチプロセス実行→次の関数の下で実行されている
def df_parallelize_run(func, t_split):
    #N_CORESでcpuのコア数を表示することができる
    #cpuを倉庫だとするとコア数っていうのは受け取った資材を処理する装置。これが多いと複数のデータを並列処理することが可能である
    num_cores = np.min([N_CORES,len(t_split)])
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, t_split), axis=1)
    pool.close()
    pool.join()
    return df

#売り上げに対してのlagの特徴量を作る関数
def make_normal_lag(lag_day):
    lag_df = grid_df[['id','d',TARGET]] 
    col_name = 'sales_lag_'+str(lag_day)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(lag_day)).astype(np.float16)
    return lag_df[[col_name]]

# Launch parallel lag creation
# and "append" to our grid
LAGS_SPLIT = [col for col in range(1,1+7)]
#1週間分のlag特徴量を作成して結合する
grid_df = pd.concat([grid_df, df_parallelize_run(make_normal_lag,LAGS_SPLIT)], axis=1)

# lightgbmで使うdataframeを用意する関数
test_model = make_fast_test(grid_df)

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[378]	training's rmse: 2.5463	valid_1's rmse: 2.25816


# ここに詳細が書いてあるのでこれを確認するのが
https://www.kaggle.com/dansbecker/permutation-importance

In [28]:
########################### 順列重要度テスト　###########################

# features_columnsはlightgbmに使われる特徴量を集めたものである。remove_featuresは['id','d',TARGET]なので
features_columns = [col for col in list(grid_df) if col not in remove_features]
#さっきlightgbmのところでも作成したvalidationのdataframe
validation_df = grid_df[grid_df['d']>(END_TRAIN-28)].reset_index(drop=True)

# 予想が出来たデータを保存する
validation_df['preds'] = test_model.predict(validation_df[features_columns])
#誤差が一番小さくなるときを求める
base_score = rmse(validation_df[TARGET], validation_df['preds'])
print('Standart RMSE', base_score)


# 分析の説明変数として使われたものをまとめてfor文で出力する
for col in features_columns:
    
    # 検証セットのコピーを作成して復元します
    # 各実行の状態を特徴とする
    temp_df = validation_df.copy()
    
    # 「カテゴリ」機能があり、カテゴリを中断せずにnp.random.permutationを実行できない場合、ここでエラーが表示されます
    #  したがって、特徴が数値であるかどうかを確認する必要があります
    if temp_df[col].dtypes.name != 'category':
        #random.permutationで配列をランダムに並び替え
        #→正常のときとランダムにしたときの目的変数との相関関係を調べることで説明変数の重要度が測れるってこと
        temp_df[col] = np.random.permutation(temp_df[col].values)
        temp_df['preds'] = test_model.predict(temp_df[features_columns])
        cur_score = rmse(temp_df[TARGET], temp_df['preds'])
        
        #現在のrmseスコアがベーススコアより小さい場合それはおそらくその機能が悪いものであることを意味します
        #私たちのモデルはノイズについて学習しています
        #小数点以下4桁まで表示する
        print(col, np.round(cur_score - base_score, 4))

# 要らないデータの削除
del temp_df, validation_df

# Remove test features
#lagに関する特徴量だけを削除する対象とする
keep_cols = [col for col in list(grid_df) if 'sales_lag_' not in col]
grid_df = grid_df[keep_cols]


# 結果
## shift1はめっちゃ大事な変数である
## 他のいくつかの機能は重要ではなく、おそらくノイズである
## 機能の役に立たないことを確認するためにいくつかの順列実行を行う方が良い
## price_nunique -0.002 : 強い負の値はおそらくノイズです
## price_max -0.0002 : 0に近い値はより深い調査が必要です


Standart RMSE 2.2581550604159593
release 0.0
sell_price 0.0047
price_max 0.0008
price_min 0.0003
price_std 0.0029
price_mean 0.0017
price_norm 0.0135
price_nunique 0.0003
item_nunique -0.0002
price_momentum 0.0001
price_momentum_m 0.0048
price_momentum_y 0.0015
tm_d 0.0097
tm_w 0.0003
tm_m 0.0002
tm_y 0.0
tm_wm 0.0001
tm_dw 0.1387
tm_w_end 0.0099
sales_lag_1 0.5717
sales_lag_2 0.0461
sales_lag_3 0.0213
sales_lag_4 0.0153
sales_lag_5 0.0201
sales_lag_6 0.019
sales_lag_7 0.0363


アイデアは次のとおりです。機能の重要性は、機能が利用できない場合にスコア（正確さ、mse、rmse、maeなど-興味のあるスコア）がどれだけ減少するかを調べることで測定できます。

そのためには、データセットから特徴を削除し、推定器を再トレーニングしてスコアを確認します。  
ただし、機能ごとに推定器を再トレーニングする必要があるため、計算量が多くなる可能性があります。  
また、トレーニング済みの具体的なモデルでは何が重要であるかではなく、データセット内で何が重要かを示します。

推定器の再トレーニングを回避するために、データセットのテスト部分からのみ機能を削除し、この機能を使用せずにスコアを計算できます。  
推定者は機能が存在することを期待しているため、現状では機能しません。  
したがって、フィーチャを削除する代わりに、**ランダムノイズで置き換える**ことができます。  
フィーチャ列はまだ残っていますが、有用な情報は含まれていません。

この方法は、**元の特徴値と同じ分布**からノイズが抽出された場合に機能します（そうしないと、推定器が失敗する可能性があります）。  
このようなノイズを取得する最も簡単な方法は、特徴の値をシャッフルすることです。  
つまり、他の例の特徴値を使用します。これが順列の重要度を計算する方法です。

---

# 機能が削除された場合（ノイズに置き換えられた場合）は適切ではありませんが、スコアは高くなります。シンプルで簡単。
lagshiftの特徴量が重要なものであるとわかったので試しに2ヶ月後をやってみましたっていうこと  
→lagの日数を変えた以外はほとんどやっていることは同じ

In [29]:
########################### Lets test far away Lags (7 days with 56 days shift)　###########################
########################### and check permutation importance　###########################

#　2ヶ月後のデータも付け加えている
LAGS_SPLIT = [col for col in range(56,56+7)]
#make_normal_lagは売り上げに対してのlagの特徴量を作る関数
#df_parallelize_runは並列処理が出来る関数
grid_df = pd.concat([grid_df, df_parallelize_run(make_normal_lag,LAGS_SPLIT)], axis=1)
test_model = make_fast_test(grid_df)

#これ何度目だよ→特徴量にlagの56日分を加えてたので再度分析をしてるって訳だと思う
features_columns = [col for col in list(grid_df) if col not in remove_features]
validation_df = grid_df[grid_df['d']>(END_TRAIN-28)].reset_index(drop=True)
validation_df['preds'] = test_model.predict(validation_df[features_columns])
base_score = rmse(validation_df[TARGET], validation_df['preds'])
print('Standart RMSE', base_score)

#これも上でやっているのと同じ処理
for col in features_columns:
    temp_df = validation_df.copy()
    if temp_df[col].dtypes.name != 'category':
        temp_df[col] = np.random.permutation(temp_df[col].values)
        temp_df['preds'] = test_model.predict(temp_df[features_columns])
        cur_score = rmse(temp_df[TARGET], temp_df['preds'])
        print(col, np.round(cur_score - base_score, 4))

del temp_df, validation_df
        
# Remove test features
# As we will compare performance with baseline model for now
keep_cols = [col for col in list(grid_df) if 'sales_lag_' not in col]
grid_df = grid_df[keep_cols]


# Results:
# 56日のシフトがあるラグ（過去の遠い）はそれほど重要ではなく、最も近い過去のラグほど重要ではありません。
# ある時点では、モデルにとって単なるノイズになります。

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[261]	training's rmse: 2.86081	valid_1's rmse: 2.4008
Standart RMSE 2.400802140889982
release 0.0
sell_price 0.015
price_max 0.0068
price_min 0.0056
price_std 0.0069
price_mean 0.009
price_norm 0.0589
price_nunique 0.0266
item_nunique 0.0077
price_momentum 0.0001
price_momentum_m 0.0361
price_momentum_y 0.0084
tm_d 0.0055
tm_w 0.0021
tm_m 0.0
tm_y 0.0
tm_wm -0.0001
tm_dw 0.1152
tm_w_end 0.0113
sales_lag_56 0.0325
sales_lag_57 0.0097
sales_lag_58 0.0042
sales_lag_59 0.0032
sales_lag_60 0.0009
sales_lag_61 0.0048
sales_lag_62 0.0066


In [30]:
########################### PCA ###########################

# PCA は教師なし学習
# ターゲットがシフトしているので、ターゲットリークがないことを確認できます
from sklearn.decomposition import PCA

#後々pca_colには'id'が入る。n_days=7でこの分だけsalesのlag特徴量が作成される
def make_pca(df, pca_col, n_days):
    print('PCA:', pca_col, n_days)
    
    # 主成分分析には使用しない変数をまとめておく
    pca_df = df[[pca_col,'d',TARGET]]
    
    # 変数を絞っていく作業
    if pca_col != 'id':
        merge_base = pca_df[[pca_col,'d']]
        pca_df = pca_df.groupby([pca_col,'d'])[TARGET].agg(['sum']).reset_index()
        pca_df[TARGET] = pca_df['sum']
        del pca_df['sum']
    
    #最大値によるスケーリングを行う
    pca_df[TARGET] = pca_df[TARGET]/pca_df[TARGET].max()
    
    # n_daysは関数で指定をしている
    #今回は下のほうで7と指定をしているのでlag1~7まで作成をされる
    LAG_DAYS = [col for col in range(1,n_days+1)]
    format_s = '{}_pca_'+pca_col+str(n_days)+'_{}'
    #assignによって作成した特徴量を結合できる
    pca_df = pca_df.assign(**{
            format_s.format(col, l): pca_df.groupby([pca_col])[col].transform(lambda x: x.shift(l))
            for l in LAG_DAYS
            for col in [TARGET]
        })
    #'id'と'd'は使わないので削除してしまうってことなはず
    pca_columns = list(pca_df)[3:]
    #欠損値を0で穴埋めする→PCAは欠損地があるとできないため
    pca_df[pca_columns] = pca_df[pca_columns].fillna(0)
    #実際に主成分分析の設定を行う
    pca = PCA(random_state=SEED)
    
    #ここで実際に変換を行う
    pca.fit(pca_df[pca_columns])
    pca_df[pca_columns] = pca.transform(pca_df[pca_columns])
    print(pca.explained_variance_ratio_)
    
    # 主成分分析で3次元までは残すってことなはず
    keep_cols = pca_columns[:3]
    print('Columns to keep:', keep_cols)
    return pca_df[keep_cols]


# 主成分分析によって特徴量を絞る
grid_df = pd.concat([grid_df, make_pca(grid_df,'id',7)], axis=1)

# make_fast_testでlightgbmで使うdataframeを用意して分析まで行う関数
test_model = make_fast_test(grid_df)

# Remove test features
# As we will compare performance with baseline model for now
keep_cols = [col for col in list(grid_df) if '_pca_' not in col]
grid_df = grid_df[keep_cols]

PCA: id 7
[0.72224136 0.06621842 0.05938444 0.04201445 0.03891686 0.03614344
 0.03508102]
Columns to keep: ['sales_pca_id7_1', 'sales_pca_id7_2', 'sales_pca_id7_3']
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[292]	training's rmse: 2.64421	valid_1's rmse: 2.26395


In [31]:
########################### Mean/std target encoding ########################### 

# targetencodingをするのにこの３つを使うらしい
icols = ['item_id','cat_id','dept_id']

#地味にtarget encodingを標準偏差と平均値によって行っている
# icols = ['item_id','cat_id','dept_id']
for col in icols:
    print('Encoding', col)
    temp_df = grid_df[grid_df['d']<=(1941-28)] # target encodingをやる際にはデータがleakageしないように注意をしないといけない
    #aggの中にtargetを入れて分析をするんだと！感心しました！店ごとに区切ることで直接的にleakageをしないようにしているってことだと思う。
    temp_df = temp_df.groupby([col,'store_id']).agg({TARGET: ['std','mean']})
    joiner = '_'+col+'_encoding_'
    #この行の扱い方がイマイチわからない。なにをどうstripしているのか？？
    temp_df.columns = [joiner.join(col).strip() for col in temp_df.columns.values]
    temp_df = temp_df.reset_index()
    grid_df = grid_df.merge(temp_df, on=[col,'store_id'], how='left')
    del temp_df

# make_fast_testでlightgbmで使うdataframeを用意して分析まで行う関数
test_model = make_fast_test(grid_df)

# Remove test features
keep_cols = [col for col in list(grid_df) if '_encoding_' not in col]
grid_df = grid_df[keep_cols]

Encoding item_id
Encoding cat_id
Encoding dept_id
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[404]	training's rmse: 2.77462	valid_1's rmse: 2.39129


In [32]:
########################### Last non O sale　###########################

def find_last_sale(df,n_day):
    
    # Limit initial df
    ls_df = df[['id','d',TARGET]]
    
    # 不等式にすることでいったんtrueかfalseかで表し、intにすることで0,1表記にしている。
    ls_df['non_zero'] = (ls_df[TARGET]>0).astype(np.int8)
    
    #lagを作成することでleakageを防いでいるらしい
    #rollingの中の(2000,1)がよくわからない。
    #fillna(-1)という-1はありえない値なのでこれを入れることでうまくデータを制御しているらしいぞ
    #売れたか売れていないかの0,1の合計で説明変数を取ろうとしている
    ls_df['non_zero_lag'] = ls_df.groupby(['id'])['non_zero'].transform(lambda x: x.shift(n_day).rolling(2000,1).sum()).fillna(-1)

    temp_df = ls_df[['id','d','non_zero_lag']].drop_duplicates(subset=['id','non_zero_lag'])
    temp_df.columns = ['id','d_min','non_zero_lag']

    ls_df = ls_df.merge(temp_df, on=['id','non_zero_lag'], how='left')
    ls_df['last_sale'] = ls_df['d'] - ls_df['d_min']

    return ls_df[['last_sale']]


# Find last non zero
# Need some "dances" to fit in memory limit with groupers
grid_df = pd.concat([grid_df, find_last_sale(grid_df,1)], axis=1)

# Make features test
test_model = make_fast_test(grid_df)

# Remove test features
keep_cols = [col for col in list(grid_df) if 'last_sale' not in col]
grid_df = grid_df[keep_cols]

Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[337]	training's rmse: 2.67507	valid_1's rmse: 2.299


In [33]:
########################### Apply on grid_df
#################################################################################
# lets read grid from 
# https://www.kaggle.com/kyakovlev/m5-simple-fe
# to be sure that our grids are aligned by index
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df[TARGET][grid_df['d']>(1941-28)] = np.nan
base_cols = list(grid_df)

icols =  [
            ['state_id'],
            ['store_id'],
            ['cat_id'],
            ['dept_id'],
            ['state_id', 'cat_id'],
            ['state_id', 'dept_id'],
            ['store_id', 'cat_id'],
            ['store_id', 'dept_id'],
            ['item_id'],
            ['item_id', 'state_id'],
            ['item_id', 'store_id']
            ]
#こんなになんパターンもencodingってできるんだという感想
for col in icols:
    print('Encoding', col)
    col_name = '_'+'_'.join(col)+'_'
    grid_df['enc'+col_name+'mean'] = grid_df.groupby(col)[TARGET].transform('mean').astype(np.float16)
    grid_df['enc'+col_name+'std'] = grid_df.groupby(col)[TARGET].transform('std').astype(np.float16)

keep_cols = [col for col in list(grid_df) if col not in base_cols]
grid_df = grid_df[['id','d']+keep_cols]

Encoding ['state_id']
Encoding ['store_id']
Encoding ['cat_id']
Encoding ['dept_id']
Encoding ['state_id', 'cat_id']
Encoding ['state_id', 'dept_id']
Encoding ['store_id', 'cat_id']
Encoding ['store_id', 'dept_id']
Encoding ['item_id']
Encoding ['item_id', 'state_id']
Encoding ['item_id', 'store_id']


In [34]:
#################################################################################
print('Save Mean/Std encoding')
grid_df.to_pickle('mean_encoding_df.pkl')

Save Mean/Std encoding


In [35]:
########################### Final list of new features
#################################################################################
grid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47735397 entries, 0 to 47735396
Data columns (total 24 columns):
id                           category
d                            int16
enc_state_id_mean            float16
enc_state_id_std             float16
enc_store_id_mean            float16
enc_store_id_std             float16
enc_cat_id_mean              float16
enc_cat_id_std               float16
enc_dept_id_mean             float16
enc_dept_id_std              float16
enc_state_id_cat_id_mean     float16
enc_state_id_cat_id_std      float16
enc_state_id_dept_id_mean    float16
enc_state_id_dept_id_std     float16
enc_store_id_cat_id_mean     float16
enc_store_id_cat_id_std      float16
enc_store_id_dept_id_mean    float16
enc_store_id_dept_id_std     float16
enc_item_id_mean             float16
enc_item_id_std              float16
enc_item_id_state_id_mean    float16
enc_item_id_state_id_std     float16
enc_item_id_store_id_mean    float16
enc_item_id_store_id_std     float1